In [30]:
import pandas as pd
from pathlib import Path

In [31]:
ddf = pd.read_csv('../data/curated/historical_data.csv')
popudf = pd.read_csv('../data/curated/subpopu.csv')
popudf = popudf[['POSTCODE', '2016', '2017', '2018', '2019', '2020']]

ddf = ddf.rename(columns={'Sub': 'suburb'})
incomedf = pd.read_csv('../data/curated/subincome.csv')
incomedf = incomedf[['POSTCODE', '2016', '2017', '2018', '2019', '2020']]

In [32]:
years = ['2016', '2017', '2018', '2019', '2020']

new_popudf = pd.DataFrame(columns=['POSTCODE', 'Year', 'population'])
for year in years:
    df = popudf[['POSTCODE', year]]
    df = df.rename(columns={year: 'population'})
    df['Year'] = year
    new_popudf = pd.concat([new_popudf, df], axis=0)

In [33]:
new_incomedf = pd.DataFrame(columns=['POSTCODE', 'Year', 'income'])
for year in years:
    df = incomedf[['POSTCODE', year]]
    df = df.rename(columns={year: 'income'})
    df['Year'] = year
    new_incomedf = pd.concat([new_incomedf, df], axis=0)

In [34]:
df_merged = pd.merge(new_incomedf, new_popudf, on=['POSTCODE', 'Year'], how='inner')

In [35]:
sub_postcodeinfo = pd.read_csv('../data/raw/realestate.csv')
sub_postcodeinfo = sub_postcodeinfo[['suburb', 'postcode']]
sub_postcode = sub_postcodeinfo.groupby(['suburb', 'postcode']).count().reset_index()
sub_postcode = sub_postcode.rename(columns={'postcode': 'POSTCODE'})

In [36]:
df_merged = pd.merge(df_merged, sub_postcode, on='POSTCODE', how='inner')
df_merged = df_merged[['suburb', 'Year', 'population', 'income']]
df_merged=df_merged.groupby(['suburb', 'Year']).agg({'population': 'sum',
                                                     'income': 'mean'}).reset_index()


In [37]:
df_merged['Year'] = df_merged['Year'].astype(int)
df_merged = pd.merge(df_merged, ddf, on=['Year', 'suburb'], how='inner')

In [38]:
filepath = Path('../data/curated/ques2data.csv')
df_merged.to_csv(filepath, index=False)